In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('data/train.csv', index_col='id')
train.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
train.shape

(20800, 4)

In [4]:
train.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
train = train.fillna(' ')
train['total'] = train['title'] + ' ' + train['author'] + train['text']
train.head()

,title,author,text,label,total
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [6]:
train.isnull().sum()

title     0
author    0
text      0
label     0
total     0
dtype: int64

In [7]:
df = train.drop(['title', 'author', 'total'], axis=1)
df.head()

,text,label
id,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
import re
import string
def wordopt(text):
    text = str(text)
    text = text.lower()
    text = re.sub("\[.*?\]", "", text)
    text = re.sub("\\W", " ", text)
    text = re.sub("https?://\S+|www\.\S+", "", text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [9]:
df['text'] = df['text'].apply(wordopt)

In [10]:
df.head()

,text,label
id,,
0,house dem aide we didn t even see comey s let...,1
1,ever get the feeling your life circles the rou...,0
2,why the truth might get you fired october ...,1
3,videos civilians killed in single us airstrik...,1
4,print an iranian woman has been sentenced to ...,1


In [11]:
X = df['text']
y = df['label']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15600,)
(5200,)
(15600,)
(5200,)


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

tfidf_test = tfidf_vectorizer.transform(X_test)

In [15]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB() 
MNB.fit(tfidf_train, y_train)
pred = MNB.predict(tfidf_test)
from sklearn.metrics import confusion_matrix,accuracy_score
MNBcm = confusion_matrix(y_test, pred)
print(MNBcm)
MNBscore = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % MNBscore)

[[2601   41]
 [ 665 1893]]
accuracy:   0.864


In [16]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(random_state = 0)
LR.fit(tfidf_train, y_train )
pred = LR.predict(tfidf_test)
LRcm = confusion_matrix(y_test, pred)
print(LRcm)
LRscore = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % LRscore)

[[2453  189]
 [ 132 2426]]
accuracy:   0.938


In [17]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 1000)
KNN.fit(tfidf_train, y_train)
pred = KNN.predict(tfidf_test)
KNNcm = confusion_matrix(y_test, pred)
print(KNNcm)
KNNscore = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % KNNscore)

[[2332  310]
 [ 405 2153]]
accuracy:   0.863


In [18]:
from sklearn.linear_model import PassiveAggressiveClassifier
PA = PassiveAggressiveClassifier(random_state=0)
PA.fit(tfidf_train, y_train)
pred = PA.predict(tfidf_test)
PAcm = confusion_matrix(y_test, pred)
print(PAcm)
PAscore = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % PAscore)

[[2527  115]
 [ 117 2441]]
accuracy:   0.955


In [19]:
print("MultinomialNB\n", MNBcm)
print("\nLogisticRegression\n", LRcm)
print("\nKNeighborsClassifier\n", KNNcm)
print("\nPassiveAggressiveClassifier", PAcm)

MultinomialNB
 [[2601   41]
 [ 665 1893]]

LogisticRegression
 [[2453  189]
 [ 132 2426]]

KNeighborsClassifier
 [[2332  310]
 [ 405 2153]]

PassiveAggressiveClassifier [[2527  115]
 [ 117 2441]]


In [20]:
data = [[1, "MultinomialNB", MNBscore],
        [2, "LogisticRegression", LRscore],
        [3, "KNeighborsClassifier", KNNscore],
        [4, "PassiveAggressiveClassifier", PAscore]]
headers = ["S.no.", "Classifier", "Score"]
results = pd.DataFrame(data, columns=headers)
results.set_index("S.no.", inplace=True)
results

,Classifier,Score
S.no.,,
1,MultinomialNB,0.864231
2,LogisticRegression,0.938269
3,KNeighborsClassifier,0.862500
4,PassiveAggressiveClassifier,0.955385


In [21]:
import pickle

model_filename = '../pickles/PA.pickle'
vectorizer_filename = '../pickles/tfidf_vectorizer.pickle'
pickle.dump(PA, open(model_filename, 'wb'))
pickle.dump(tfidf_vectorizer, open(vectorizer_filename, 'wb'))